# Random Forest from Scratch

## 1. Introduction to Random Forest

Random Forest is an ensemble learning method for classification and regression that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

It addresses the problem of overfitting that individual decision trees often suffer from, and it generally provides higher accuracy and stability.

Key concepts that make Random Forest powerful:
- **Ensemble Learning**: Combining multiple models to improve overall performance.
- **Decision Trees**: The fundamental building blocks of a Random Forest.
- **Bagging (Bootstrap Aggregating)**: A technique to reduce variance by training multiple models on different subsets of the training data.
- **Random Feature Subspace**: Introducing randomness in feature selection when building individual trees.

## 2. Decision Trees (Building Blocks)

A decision tree is a flowchart-like structure where each internal node represents a "test" on an attribute (e.g., whether a coin flip comes up heads or tails), each branch represents the outcome of the test, and each leaf node represents a class label (decision taken after computing all attributes). The paths from root to leaf represent classification rules.

**Key Concepts:**
- **Root Node**: The topmost node in the tree, representing the entire dataset.
- **Internal Node**: A node that has one or more branches, representing a decision based on a feature.
- **Leaf Node (Terminal Node)**: A node that does not have any branches, representing the final outcome or prediction.
- **Splitting**: The process of dividing a node into two or more sub-nodes based on a splitting criterion.
- **Pruning**: The process of removing branches from the tree to prevent overfitting.

**How a Decision Tree is Built (Simplified):**
1.  Start with the entire dataset at the root node.
2.  Select the best attribute to split the data based on a criterion (e.g., Gini impurity, entropy for classification; Mean Squared Error for regression).
3.  Divide the dataset into subsets based on the values of the chosen attribute.
4.  Recursively repeat steps 2 and 3 for each subset until a stopping condition is met (e.g., all samples in a node belong to the same class, maximum depth is reached, or minimum number of samples per leaf is met).

**Mathematical Intuition (for Classification - Gini Impurity):**

Gini Impurity measures the impurity of a node. A node is pure if all its samples belong to the same class (Gini impurity = 0). The goal is to minimize Gini impurity at each split.

$Gini(D) = 1 - \sum_{i=1}^{c} (p_i)^2$

Where:
- $D$ is the dataset (or a node).
- $c$ is the number of classes.
- $p_i$ is the proportion of samples belonging to class $i$ in the dataset $D$.

When splitting a node into two child nodes (left and right), the weighted average Gini impurity is calculated:

$Gini_{split} = \frac{N_{left}}{N} Gini(D_{left}) + \frac{N_{right}}{N} Gini(D_{right})$

Where:
- $N_{left}$ and $N_{right}$ are the number of samples in the left and right child nodes, respectively.
- $N$ is the total number of samples in the parent node.

The attribute that results in the lowest $Gini_{split}$ (or highest information gain) is chosen for the split.

## 3. Bagging (Bootstrap Aggregating)

Bagging, short for Bootstrap Aggregating, is an ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms. It also helps to reduce variance and avoid overfitting.

**How it works:**
1.  **Bootstrap Sampling**: From the original training dataset, multiple subsets are created by sampling with replacement. This means that some data points may appear multiple times in a subset, while others may not appear at all. Each subset is of the same size as the original dataset.
2.  **Model Training**: An independent model (e.g., a decision tree) is trained on each of these bootstrap samples.
3.  **Aggregation**: For classification tasks, the final prediction is made by taking a majority vote of the predictions from all individual models. For regression tasks, the final prediction is the average of the predictions from all individual models.

**Benefits of Bagging:**
-   **Reduces Variance**: By averaging or voting across multiple models trained on different data subsets, the impact of noisy data or outliers on any single model is reduced.
-   **Improves Stability**: The overall model becomes more robust to small changes in the training data.
-   **Handles Overfitting**: By introducing randomness in the data sampling, it helps to prevent individual models from overfitting to the training data.

## 4. Random Feature Subset Selection

While bagging helps reduce variance by sampling data, Random Forest introduces an additional layer of randomness: **random feature subset selection**.

**How it works:**
When building each individual decision tree in the forest, at each split point, instead of considering all available features, only a random subset of features is considered. This means that even if there's a very strong predictor feature, not all trees will be built using it, forcing them to explore other features.

**Benefits:**
-   **Decorrelates Trees**: This is the most crucial benefit. If you have one or a few very strong predictor features, these features would be chosen at the top of almost every tree in a standard bagging approach, leading to highly correlated trees. Averaging correlated trees doesn't reduce variance as much as averaging uncorrelated trees. By randomly selecting a subset of features, the trees become more diverse and less correlated.
-   **Reduces Overfitting**: By limiting the features available at each split, it further reduces the chance of individual trees overfitting to specific features in the training data.
-   **Computational Efficiency**: For datasets with a very large number of features, considering only a subset of features at each split can speed up the tree building process.

## 5. Algorithm Steps

Here's a step-by-step breakdown of how the Random Forest algorithm works:

1.  **Input**: A training dataset $D = \{(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)\}$ and the number of trees to build $T$.

2.  **For each tree $t$ from $1$ to $T$:**
    a.  **Bootstrap Sampling**: Create a bootstrap sample $D_t$ by randomly sampling $n$ data points from $D$ with replacement. This $D_t$ will be the training data for tree $t$.
    b.  **Tree Construction**: Grow a decision tree $C_t$ from the bootstrap sample $D_t$. During the tree growing process, at each split:
        i.  **Random Feature Subset Selection**: Randomly select a subset of $m$ features from the total $M$ available features ($m \ll M$).
        ii. **Best Split**: Find the best split among these $m$ features using a splitting criterion (e.g., Gini impurity for classification, Mean Squared Error for regression).
        iii. **Split Node**: Split the node into two child nodes based on the best split.
        iv. **Repeat**: Recursively repeat steps i-iii until a stopping condition is met (e.g., maximum depth, minimum samples per leaf, or no further gain from splitting).

3.  **Output**: The ensemble of $T$ decision trees ${C_1, C_2, ..., C_T}$.

**Prediction Phase:**

To make a prediction for a new, unseen data point $x_{new}$:

1.  **Individual Predictions**: Pass $x_{new}$ through each of the $T$ decision trees to get $T$ individual predictions: $P_1, P_2, ..., P_T$.

2.  **Aggregation**:
    a.  **For Classification**: The final prediction is the class that receives the majority vote among all $T$ trees.
    b.  **For Regression**: The final prediction is the average of the predictions from all $T$ trees.

## 6. Implementation from Scratch

We will implement a simplified version of Random Forest from scratch. This will involve creating a `DecisionTree` class and then a `RandomForest` class that utilizes multiple `DecisionTree` instances.

In [1]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # For leaf nodes

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # Check stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_feature, best_threshold = self._best_split(X, y, feat_idxs)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left_child = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_child = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_threshold, left_child, right_child)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])

        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _split(self, X_column, threshold):
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()
        return left_idxs, right_idxs

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [2]:
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_features=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_features=self.n_features,
            )
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Transpose to have predictions for each sample in columns
        tree_preds = np.swapaxes(predictions, 0, 1)
        # Get majority vote for each sample
        y_pred = np.array([self._most_common_label(pred) for pred in tree_preds])
        return y_pred

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

## 7. Example Usage

Let's test our Random Forest implementation with a simple example.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Initialize and train the Random Forest
# You can adjust n_trees, max_depth, and n_features
clf = RandomForest(n_trees=20, max_depth=10, n_features=5)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

print(f"Random Forest Accuracy: {accuracy(y_test, predictions)}")